In [1]:
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import optimizers
import keras
import pandas
import utils

Using TensorFlow backend.


In [9]:
train_folder = 'data/train'
test_folder = 'data/test'
validation_folder = 'data/validation'
labels_csv = 'train_labels.csv'
label_names = ['0', '1']
IMG_WIDTH = 450
IMG_HEIGHT = 450
batch_size = 2

In [3]:
utils.create_labelled_folders(train_folder, labels_csv, label_names)        
utils.create_validation_subfolders(train_folder, validation_folder, label_names, 0.2)        

Folders are already labelled


In [10]:
train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    rescale=1./255.,
    vertical_flip=True,
    horizontal_flip=True,
    rotation_range=90
)

validation_datagen = ImageDataGenerator(
    rescale=1./255.
)

train_generator = train_datagen.flow_from_directory(
    train_folder,
    batch_size=batch_size,
    target_size = (IMG_WIDTH, IMG_HEIGHT)
)

validation_generator = validation_datagen.flow_from_directory(
    validation_folder,
    batch_size=batch_size,
    target_size = (IMG_WIDTH, IMG_HEIGHT)
)

Found 1837 images belonging to 2 classes.
Found 458 images belonging to 2 classes.


In [15]:
base_model = keras.applications.vgg16.VGG16(weights='imagenet', 
                                            include_top=False, 
                                            input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), 
                                            classes=2)

In [16]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 450, 450, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 450, 450, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 450, 450, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 225, 225, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 225, 225, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 225, 225, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 112, 112, 128)     0         
__________

In [22]:
for layer in base_model

'block5_conv3'

In [13]:
x = base_model.output
x = (Flatten(input_shape=base_model.output_shape[1:], name="finetune"))(x)
x = (Dense(256, activation='relu'))(x)
x = (Dropout(0.5))(x)
x = (Dense(2, activation='sigmoid'))(x)

# model = Model(inputs=base_model.inputs, outputs=x)

model = base_model 
# for layer in model.layers:
#     if layer.name == "finetune":
#         break
    
#     layer.trainable = False

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [14]:
num_training_samples = train_generator.n
num_validation_samples = validation_generator.n

model.fit_generator(
    train_generator,
    steps_per_epoch=num_training_samples//batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=num_validation_samples//batch_size)

Epoch 1/50


ValueError: Error when checking target: expected avg_pool to have 4 dimensions, but got array with shape (2, 2)

In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1./255.
)

test_generator = test_datagen.flow_from_directory(
    test_folder,
    batch_size=batch_size,
    target_size = (IMG_WIDTH, IMG_HEIGHT)
)

In [ ]:
num_test_samples = test_generator.n
test_predictions = model.predict_generator(test_generator, num_test_samples//batch_size)

In [ ]:
test_predictions[:,:20]